In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option("display.max_columns", None)

2025-04-25 10:01:16.742 | INFO     | inventario_finanzas_int.config:<module>:11 - PROJ_ROOT path is: C:\Users\jarojas\Desktop\Javier\INT\inventario-finanzas-int


In [ ]:
df_mobiliario_consolidado = pd.read_excel(
    "../data/raw/MOBILIARIO (ARIEL)/MATRIZ CORRELATIVA AÑO 2025.xlsx"
)

In [5]:
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")

# Lee Informaticos
df_informaticos = pd.read_csv("../data/processed/df_procesada_informaticos.csv")

In [9]:
union_bienes_con_informatico = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)
union_bienes_con_informatico["conteo"] = 1

union_bienes_sin_informatico = pd.concat([df_mobiliario, df_equipos_medicos, df_industriales])
union_bienes_sin_informatico["conteo"] = 1

In [13]:
conteo_equipos_con_informatico = pd.pivot_table(
    union_bienes_con_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

conteo_equipos_sin_informatico = pd.pivot_table(
    union_bienes_sin_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

## Asignacion de Correlativos

Se hara de forma separada

In [15]:
GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO = ["INT", "SSMO", "MINSAL"]
GLOSAS_INDUSTRIALES_CON_CORRELATIVO = ["INT", "MEL", "DONACION"]

In [31]:
# Identifica los registros a los que hay que asignar correlativo
mask_correlativo_medicos = df_equipos_medicos["propiedad"].isin(
    GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO
)
df_equipos_medicos_con_correlativos = df_equipos_medicos[mask_correlativo_medicos]

# Asigna correlativos a los registros que requieren correlativo
correlativos_equipos_medicos = [
    f"2025-{i}" for i in range(4016, 4016 + len(df_equipos_medicos_con_correlativos))
]
df_equipos_medicos.loc[mask_correlativo_medicos, "correlativo_2025"] = correlativos_equipos_medicos

In [32]:
df_equipos_medicos

,correlativo_antiguo,bien,marca,modelo,serie,unidadservicio_clinico,ubicacion_unidad,piso,propiedad,unidad_hoja,tipo_bien,correlativo_2025
0,3-6555,CICLOERGOMETRO,MONARK,ERGOMETRICA,9035082,MEDICINA FISICA Y REHABILITACION,GIMNASIO,-1.0,INT,MQ 3 NORTE,EQUIPO MEDICO,2025-4016
1,SIN INVENTARIO,ANDADOR CON RUEDAS,BLUDING,AD-03,NaN,MEDICINA FISICA Y REHABILITACION,GIMNASIO,-1.0,INT,MQ 3 NORTE,EQUIPO MEDICO,2025-4017
2,SIN INVENTARIO,CAMILLA DE EXAMEN,NaN,NaN,NaN,CONSULTORIO EXTERNO,BOX 10,1.0,INT,CONSUL. EXTERNO,EQUIPO MEDICO,2025-4018
3,SIN INVENTARIO,ESCABEL DOS PELDANO,NaN,NaN,NaN,CONSULTORIO EXTERNO,BOX 10,1.0,INT,CONSUL. EXTERNO,EQUIPO MEDICO,2025-4019
4,1499-2176,CAMILLA DE EXAMEN,NaN,NaN,NaN,CONSULTORIO EXTERNO,BOX 11,1.0,INT,CONSUL. EXTERNO,EQUIPO MEDICO,2025-4020
...,...,...,...,...,...,...,...,...,...,...,...,...
968,3-14760,MONITOR MULTIPARAMETROS (EQUIPOS MEDICOS),PHILIPS,MX600,DE12503143,UPC 5 NORTE,UNIDAD DE EQUIPOS MEDICOS,5.0,INT,UPC 5 NORTE,EQUIPO MEDICO,2025-4889
969,1499-1011,MOTOR DE COLCHON ANTIESCARA (EQUIPOS MEDICOS ),CARE OF SWEDEN,CUROCELL 4AD,P1705-00395,UPC 5 NORTE,UNIDAD DE EQUIPOS MEDICOS,5.0,INT,UPC 5 NORTE,EQUIPO MEDICO,2025-4890
970,1499-3337,MOTOR DE COLCHON ANTIESCARA (EQUIPOS MEDICOS),CARE OF SWEDEN,CUROCELL A4,CP102-190828-00082,UPC 5 NORTE,UNIDAD DE EQUIPOS MEDICOS,5.0,INT,UPC 5 NORTE,EQUIPO MEDICO,2025-4891
971,VERIFICAR ETIQUETA,CONTADOR DE CELULAS,DIGISYSTEM,DBC-9D-M,BCM-22080491,LABORATORIO,LABORATORIO 24/7,NaN,COMODATO,UMT Y LAB.,EQUIPO MEDICO,nan


In [30]:
# Identifica los registros a los que hay que asignar correlativo
mask_asignar_correlativo_industriales = df_industriales["propiedad"].isin(
    GLOSAS_INDUSTRIALES_CON_CORRELATIVO
)
df_industriales.loc[mask_asignar_correlativo_industriales, "correlativo_2025"] = [
    f"2025-{i}"
    for i in range(4016, 4016 + len(df_industriales[mask_asignar_correlativo_industriales]))
]

In [26]:
df_industriales[mask_asignar_correlativo_industriales]

,correlativo_antiguo,bien,marca,modelo,serie,propiedad,piso,unidadservicio_clinico,ubicacion_unidad,observacion,tipo_bien,correalativo_2025,correlativo_2025
0,1499-5880,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,AREA SUCIA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4016,2025-4016
1,1499-5881,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,AREA SUCIA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4017,2025-4017
2,1499-5908,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,AREA LIMPIA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4018,2025-4018
3,1499-xxxx,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,PREPARACION,NO OPERATIVO,EQUIPO INDUSTRIAL,2025-4019,2025-4019
4,1499-5852,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,PREP. TRANSDUCTORES,OPERATIVO,EQUIPO INDUSTRIAL,2025-4020,2025-4020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,NaN,TOMA MURAL AIRE,GENTEC,S/D,S/D,INT,2,BRONCOSCOPIA,SALA 3,OPERATIVO,EQUIPO INDUSTRIAL,2025-4876,2025-4876
1067,NaN,TOMA MURAL OXIGENO,GENTEC,S/D,S/D,INT,2,BRONCOSCOPIA,SALA 3,OPERATIVO,EQUIPO INDUSTRIAL,2025-4877,2025-4877
1068,NaN,TOMA MURAL OXIGENO,GENTEC,S/D,S/D,INT,2,BRONCOSCOPIA,SALA 3,OPERATIVO,EQUIPO INDUSTRIAL,2025-4878,2025-4878
1069,NaN,TOMA MURAL VACIO,GENTEC,S/D,S/D,INT,2,BRONCOSCOPIA,SALA 3,OPERATIVO,EQUIPO INDUSTRIAL,2025-4879,2025-4879


In [27]:
df_industriales

,correlativo_antiguo,bien,marca,modelo,serie,propiedad,piso,unidadservicio_clinico,ubicacion_unidad,observacion,tipo_bien,correalativo_2025,correlativo_2025
0,1499-5880,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,AREA SUCIA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4016,2025-4016
1,1499-5881,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,AREA SUCIA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4017,2025-4017
2,1499-5908,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,AREA LIMPIA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4018,2025-4018
3,1499-xxxx,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,PREPARACION,NO OPERATIVO,EQUIPO INDUSTRIAL,2025-4019,2025-4019
4,1499-5852,PISTOLA DE AIRE,LEGRIS,S/D,S/D,INT,-1,ESTERILIZACION,PREP. TRANSDUCTORES,OPERATIVO,EQUIPO INDUSTRIAL,2025-4020,2025-4020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,NaN,TOMA MURAL VACIO,GENTEC,S/D,S/D,INT,2,BRONCOSCOPIA,SALA 3,OPERATIVO,EQUIPO INDUSTRIAL,2025-4879,2025-4879
1070,NaN,TOMA MURAL VACIO,GENTEC,S/D,S/D,INT,2,BRONCOSCOPIA,SALA 3,OPERATIVO,EQUIPO INDUSTRIAL,2025-4880,2025-4880
1071,NaN,HORNO ELECTRICO,S/D,S/D,S/D,FUNCIONARIO,2,BRONCOSCOPIA,SALA 2,OPERATIVO,EQUIPO INDUSTRIAL,nan,nan
1072,NaN,HERVIDOR ELECTRICO,S/D,S/D,S/D,FUNCIONARIO,2,BRONCOSCOPIA,SALA 2,OPERATIVO,EQUIPO INDUSTRIAL,nan,nan
